# Tensorflow_CIFAR10

CIFAR10은 10개의 클래스로 레이블링된 60000개의 32x32 이미지 데이터로 50000개의 Training Set과 10000개의 Test Set으로 구성되어 있다. 이 노트북에서는 CNN을 통해 해당 데이터셋에 대한 Classification을 수행할 것이다.

**References**
+ 솔라리스의 인공지능 연구실 : http://solarisailab.com/archives/2325

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets.cifar10 import load_data

## 1. 함수정의

In [25]:
# num만큼의 데이터를 랜덤하게 떼어오는 함수
def next_batch(num, data, labels):
    
  idx = np.arange(0 , len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_shuffle = [data[ i] for i in idx]
  labels_shuffle = [labels[ i] for i in idx]

  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [29]:
#중복코드를 방지하기 위한 setup 함수
def setup_conv_W(model_shape):
    return tf.Variable(tf.truncated_normal(shape=model_shape, stddev=5e-2))

def setup_conv_b(model_shape):
    return tf.Variable(tf.constant(0.1, shape=model_shape))

def setup_conv(input, W, b):
    return tf.nn.relu(tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME') + b)

## 2. 모델정의

Pooling Layer를 적용한 두 레이어와 그렇지 않은 세 레이어로 CNN을 구성하였다. 이후 Fully Connected Layer에 Softmax를 적용해 10개의 클래스로 분류하였고, Training시 Dropout을 사용했다.

In [11]:
def cnn_model(input_x):
    
    # Layer1
    W_conv1 = setup_conv_W([5,5,3,64])
    b_conv1 = setup_conv_b([64])
    conv1 = setup_conv(input_x, W_conv1, b_conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    
    # Layer2
    W_conv2 = setup_conv_W([5,5,64,64])
    b_conv2 = setup_conv_b([64])
    conv2 = setup_conv(conv1_pool, W_conv2, b_conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    
    # Layer3-5
    W_conv3 = setup_conv_W([3,3,64,128])
    b_conv3 = setup_conv_b([128])
    conv3 = setup_conv(conv2_pool, W_conv3, b_conv3)
    
    W_conv4 = setup_conv_W([3,3,128,128])
    b_conv4 = setup_conv_b([128])
    conv4 = setup_conv(conv3, W_conv4, b_conv4)
    
    W_conv5 = setup_conv_W([3,3,128,128])
    b_conv5 = setup_conv_b([128])
    conv5 = setup_conv(conv4, W_conv5, b_conv5)
    
    conv5_flat = tf.reshape(conv5, [-1,8*8*128])
    
    # Fully-Connected Layer
    W_fc1 = setup_conv_W([8*8*128,384])
    b_fc1 = setup_conv_W([384])
    fc1 = tf.nn.relu(tf.matmul(conv5_flat, W_fc1) + b_fc1)
    
    # Dropout
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
    W_fc2 = setup_conv_W([384, 10])
    b_fc2 = setup_conv_b([10])
    y_prev = tf.matmul(fc1_drop, W_fc2) + b_fc2
    y = tf.nn.softmax(y_prev)
    
    return y, y_prev

In [12]:
# 플레이스홀더
x = tf.placeholder(tf.float32, shape=[None,32,32,3])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# 데이터 다운로드
(x_train, y_train), (x_test, y_test) = load_data()
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)

In [23]:
# Loss Function, Optimizer
y, y_prev = cnn_model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_prev))
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 정확도 정의
correctness = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))

## 3. 학습세션

In [28]:
# 세션 및 학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch = next_batch(128, x_train, y_train_one_hot.eval())
        
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
            loss_print = loss.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
            
            print("Epoch : %d, Accuracy : %f, Loss Func Val : %f" % (i, train_accuracy, loss_print))
            
        sess.run(train, feed_dict={x:batch[0], y_:batch[1], keep_prob:0.8})
        
    test_accuracy = 0.0  
    for i in range(10):
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y_: test_batch[1], keep_prob: 1.0})
    test_accuracy = test_accuracy / 10;
    print("테스트 데이터 정확도: %f" % test_accuracy)

Epoch : 0, Accuracy : 0.054688, Loss Func Val : 0.018318
Epoch : 100, Accuracy : 0.093750, Loss Func Val : 0.000000


KeyboardInterrupt: 